# image alignment tests


In [1]:
import cv2
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import napari
from napari import settings

from align import *

In [2]:
#folder='E:/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/SEM Image/'
#folder='E:/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/EDS/'
"""
folder='Z:/FiB-Data/C3S + EDX/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/EDS Export/'
z_scale=192
mask_size=0.7
limit=[1,79]#[18,80]
eds_x_offset = 404.8#+22
eds_y_offset = 50.8
#expected_img_count = 63
relevant_elements = ['Ca', 'Si', 'O', 'Pt', 'C']
colormaps = ["red", "green", "blue", "yellow", "magenta"]
"""

folder='Z:/FiB-Data/C3S + EDX/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/SEM Image/' 
mask_size=0.6
z_scale=19.2
limit=[56,600]

#folder='E:/SEM Image/'
#folder='F:/2020_11_19 C3S 28d+EDX/EDS Export/'
sem_folder='F:/2020_11_19 C3S 28d+EDX/Images/SEM Image/'
#folder='D:/Avizo-Projekte/2022_01_30 CEM I 28d BIB 01-26/Images/SEM Image/'

In [3]:
folder, eds_elements = check_folder_structure(folder)

In [4]:
sem_translation, error_list, filled_canvas, loaded_images, scaling = process_translation_of_folder(folder=folder, multicore=False, do_nlm=False, mask_size=mask_size, eq_hist=True, crop_thresh=0, limit=limit )#, limit=[5,600]

loading 544 images...

trying to detect FEI scaling
SEM image saved by an FEI / thermoScientific device - probably created by FIB process
Found existing translation csv, loading...
Z:/FiB-Data/C3S + EDX/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/SEM Image/\translations.csv contains 544 lines, while 543 lines were expected
processing 544 images...
  reducing image size for translation calculations
processing image stack singlethreaded:
 processing SEM Image - SliceImage - 057.tif (1 / 544):
 processing SEM Image - SliceImage - 058.tif (2 / 544):
 processing SEM Image - SliceImage - 059.tif (3 / 544):
 processing SEM Image - SliceImage - 060.tif (4 / 544):
 processing SEM Image - SliceImage - 061.tif (5 / 544):
 processing SEM Image - SliceImage - 062.tif (6 / 544):
 processing SEM Image - SliceImage - 063.tif (7 / 544):
 processing SEM Image - SliceImage - 064.tif (8 / 544):
 processing SEM Image - SliceImage - 065.tif (9 / 544):
 processing SEM Image - SliceImage - 

In [5]:
im_cnt = len(loaded_images)
scale=(z_scale, scaling['y'], scaling['x'])

# add a shape and mark the top line of the cube

### correction of the y and x-translation

In [6]:
translation   = sem_translation.copy()
base_fn = folder + os.sep + 'translations_{}.csv'
#print(translation)
translation_csv_y    = base_fn.format('fixed_y')
translation_csv_xy   = base_fn.format('fixed_xy')
translation_manual_x = base_fn.format('manual_x')
translation_manual_y = base_fn.format('manual_y')
crop_manual          = base_fn.format('crop')


fixed_translation = load_translation_csv( translation_csv_xy, im_cnt )
final_images = None
a_list = []            
b_list = []

if len(fixed_translation) != im_cnt-1:
    #avoid the execution of following jupyter cells
    napari.settings.get_settings().application.ipy_interactive  = False
    viewer = napari.view_image(filled_canvas, name='image stack A', scale=scale)
    viewer.scale_bar.visible = True
    #viewer.add_shapes(name="select y correction line",shape_type='path', edge_width=1, edge_color='coral', face_color='royalblue', opacity=0.3)
    viewer.dims.order = (2,0,1)
    # with napari.gui_qt():
    #     viewer = napari.view_image(filled_canvas, name='select y correction line', scale=scale )
    @viewer.bind_key('y')
    def correct_y(viewer):
        global translation
        global translation_csv_y
        global translation_csv_xy
        global translation_manual_y
        print(viewer.layers['Shapes'].data[0])

        correction_dict = {}
        for point in viewer.layers['Shapes'].data[0]:
            correction_dict[math.floor(point[0])] = point[1]
        print('y', correction_dict)

        y_correction_list = get_axis_correction_list( correction_dict, im_cnt )        
        np.savetxt(translation_manual_y, np.array(y_correction_list), delimiter=",")

        for i, y in enumerate(y_correction_list):
            #print(i, y)
            #print(i, y, translation[i-1][1], translation[i-1][1] - y)
            if i>0: 
                translation[i-1][1] -= y
                #translation[i-1].append( y )
        #print(translation)
        save_translation_csv( translation, translation_csv_y )
        
        corrected_images = create_3D_stack(translation, loaded_images, False)
        viewer.add_image(corrected_images, name='image stack B', scale=scale)
        #viewer.dims.order = (1,2,0)
        viewer.layers.remove('Shapes')
        viewer.layers.remove('image stack A')

        #viewer.add_shapes(name="select x correction line", shape_type='path', edge_width=1, edge_color='red', face_color='green', opacity=0.3)

    @viewer.bind_key('x')
    def correct_x(viewer):
        global translation
        global translation_csv_xy
        global translation_manual_x
        global final_images
        #print(viewer.layers['Shapes'].data[0])

        correction_dict = {}
        for point in viewer.layers['Shapes'].data[0]:
            correction_dict[math.floor(point[0])] = point[2]
        print('x', correction_dict)

        x_correction_list = get_axis_correction_list( correction_dict, im_cnt )
        np.savetxt(translation_manual_x, np.array(x_correction_list), delimiter=",")

        for i, x in enumerate(x_correction_list):
            #print(i, x)
            #print(i, x, translation[i-1][2], translation[i-1][2] + x)
            if i>0: 
                translation[i-1][2] += x
        
        #print(translation)
        final_images = create_3D_stack(translation, loaded_images, False)
        viewer.add_image(final_images, name='Crop to Rectangle', scale=scale)
        viewer.dims.order = (0,1,2)
        viewer.layers.remove('image stack B')
        viewer.layers.remove('Shapes')
        #viewer.dims.ndisplay = 3

    @viewer.bind_key('c')
    def crop(viewer):
        global final_images
        global a_list           
        global b_list
        global crop_manual
        
        #print(viewer.layers['Shapes'].data)
        
        if len(viewer.layers['Shapes'].data[0]) == 4:
            for point in viewer.layers[1].data[0]:
                a = round(point[1])
                b = round(point[2])
                if not a in a_list: a_list.append(a)
                if not b in b_list: b_list.append(b)
        else:
            print('found less or more points than expected ({})'.format(len(viewer.layers[1].data[0])))

        a_list.sort()
        b_list.sort()
        final_images = final_images[ :, a_list[0]:a_list[1], b_list[0]:b_list[1] ]
        np.savetxt(crop_manual, np.array([a_list[0], a_list[1], b_list[0], b_list[1]]), delimiter=",")
        
        #corrected_images = create_3D_stack(translation, loaded_images, False)
        viewer.add_image(final_images, name='image stack Final', scale=scale)
        viewer.dims.order = (0,1,2)
        viewer.layers.remove('Shapes')
        viewer.layers.remove('Crop to Rectangle')
        viewer.dims.ndisplay = 3

    @viewer.bind_key('s')
    def correct_x(viewer):
        #global translation
        save_translation_csv( translation, translation_csv_xy )

        print('closing')
        viewer.close()
    
    napari.run()
    napari.settings.get_settings().application.ipy_interactive  = True
else:
    translation = fixed_translation

#print(translation)

C:\Users\Florian Kleiner\AppData\Roaming\Python\Python38\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


C:\Users\Florian Kleiner\AppData\Roaming\Python\Python38\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison!
[[ -4.61047413 748.05727631 884.        ]
 [120.12115832 687.941582   884.        ]
 [137.13001729 697.19015036 884.        ]
 [162.07634378 687.941582   884.        ]
 [218.77254034 653.25945067 884.        ]
 [230.11177966 664.82016111 884.        ]
 [272.06696512 634.76231395 884.        ]
 [308.35253092 634.76231395 884.        ]
 [370.71834714 574.64661964 884.        ]
 [452.3608702  535.34020413 884.        ]
 [505.65529497 447.47880475 884.        ]
 [506.7892189  431.29381013 884.        ]
 [527.19984966 396.61167879 884.        ]
 [544.20870863 357.30526328 884.        ]
 [553.28010009 348.05669492 884.        ]]
y {-5: 748.0572763124895, 120: 687.9415819998537, 137: 697.1901503556438, 162: 687.9415819998537, 218: 653.2594506656409, 230: 664.820161110378

## load EDS images

In [7]:

def process_element(eds_elements, selected_element, se_translation,  eds_x_offset, eds_y_offset, limit=[]):
    if selected_element in eds_elements.keys():
        print('-'*20)
        print('selected element is {}'.format(selected_element))
    else:
        print('ERROR: element {} not found in the dataset'.format(selected_element))

    #eds_each_nth_slice = 10

    folder = eds_elements[selected_element]
    images, loaded_images = load_image_set(folder, limit)
    print(images)

    image_numbering = []
    for i, image in enumerate(loaded_images):
        image_numbering.append( int(images[i].split(' ')[-1].split('.')[0]) )

    #sort lists
    images = [x for _, x in sorted(zip(image_numbering, images))]
    loaded_images = [x for _, x in sorted(zip(image_numbering, loaded_images))]
    corrected_images = loaded_images
    if len(images) > 1:
        shapes = []
        shape_counts = []
        image_numbering = []
        for i, image in enumerate(loaded_images):
            shape = image.shape
            if not shape in shapes:
                shapes.append(shape)
                shape_counts.append(0)
            shape_counts[-1] +=1

        if len(shapes) > 1:
            print( ' Found multiple shapes!' )
            for i, shape in enumerate(shapes):
                print('  found {} images with this shape:'.format(shape_counts[i]), shape)

        # TODO select the indended stack?
        ignore_first_n_images = 18#shape_counts[0]-1 if len(shapes) > 1 else 0
        ignore_last_n_images = shape_counts[-2] if len(shapes) == 3 else 0
        x_offset = 0
        y_offset = 0

        # correcting position of the eds image relative to the SE images
        if ignore_first_n_images > 0:
            t_x, t_y  = get_full_img_translation(se_translation, ignore_first_n_images)
            print('  - eds x translation: {} - {} + {} '.format(eds_x_offset, t_x[0], t_x[1]))
            print('  - eds y translation: {} - {} + {} '.format(eds_y_offset, t_y[0], t_y[1] ))
            x_offset = eds_x_offset - t_x[0] + t_x[1]
            y_offset = eds_y_offset - t_y[0] + t_y[1]
            print('  - final translation {} and {}'.format( x_offset, y_offset ) )
            print( - t_y[0] + t_y[1] )

        if ignore_last_n_images == 0:
            img_stack = loaded_images[ignore_first_n_images:]
            # select images to be displayed
            selected_translation = se_translation[ignore_first_n_images-1 :]
            #selected_translation[0] = [selected_translation[0][0], selected_translation[0][1]+eds_x_offset, selected_translation[0][2]+eds_y_offset]
        else:
            loaded_images[ignore_first_n_images : ignore_last_n_images]
            # select images to be displayed
            selected_translation = se_translation[ignore_first_n_images-1 : ignore_last_n_images]
            #selected_translation[0] = [selected_translation[0][0], selected_translation[0][1]+eds_x_offset, selected_translation[0][2]+eds_y_offset]

        print(selected_translation[0])
        #selected_translation[0] = [selected_translation[0][0], -50, -10]#y_offset]
        #selected_translation[0] = [selected_translation[0][0], selected_translation[0][1]+y_offset, selected_translation[0][2]+x_offset]
        #print(selected_translation[0])

        # process eds images to reduce noise
        print(' denoising images and enhance contrast')
        for i, img in enumerate(img_stack):
            img_stack[i] = preprocess_eds_image(img)
            #_, img_stack[i] = cv2.threshold( clahe.apply(cv2.medianBlur(img, 9)) , 50, 255, cv2.THRESH_TOZERO)#THRESH_TRUNC)
            #img_stack[i] = clahe.apply(img_stack[i])

        #corrected_images = create_3D_stack(selected_translation, img_stack, do_nlm=False, first_x_offset =49, first_y_offset =471 )
        corrected_images = create_3D_stack(selected_translation, img_stack, do_nlm=False, first_x_offset=x_offset, first_y_offset=y_offset )#52 )#x_offset = 471  y_offset should be 49-52??

        cs = corrected_images.shape
        ecs = (ignore_first_n_images + cs[0] + ignore_last_n_images, cs[1], cs[2])

        if len(shapes) > 1:
            if ignore_first_n_images > 1:
                temp = np.zeros(ecs)

                if ignore_last_n_images == 0:
                    temp[ignore_first_n_images:,:,:] = corrected_images
                else:
                    temp[ignore_first_n_images:-ignore_last_n_images,:,:] = corrected_images

                corrected_images = temp

    return corrected_images

In [10]:

napari.settings.get_settings().application.ipy_interactive  = True
#print(translation)
translation = load_translation_csv( translation_csv_xy, im_cnt )
#print(translation)
#corrected_sem_images = create_3D_stack(translation, loaded_images, False)

if len(eds_elements) > 0:

    scale=(z_scale, scaling['y'], scaling['x'])
    image_stacks = {}
    for element in eds_elements.keys():
        if element in relevant_elements:
            stack = process_element(eds_elements, element, translation, eds_x_offset, eds_y_offset, limit=limit)
            if len(stack) != len(loaded_images):
                print('Dataset for {} is incomplete ({} != {})'.format(element, len(stack), len(loaded_images)))
            else:
                image_stacks[element] = stack
        else:
            print('ignoring {}'.format(element))

    # show image stacks
    print('translate and display image stack using the following voxel dimensions: x={:.2f}, y={:.2f}, z={:.2f} {}'.format(scale[2], scale[1], scale[0], scaling['unit']))
    #scale=(z_scale*eds_each_nth_slice, x_scale/math.cos(38), x_scale)
    #with napari.gui_qt():
    viewer = napari.view_image(corrected_sem_images[ :, a_list[0]:a_list[1], b_list[0]:b_list[1] ], name='SEM images', scale=scale)#
    for i, selected_element in enumerate(image_stacks):
        print(i, selected_element)
        #new_layer = viewer.add_image(image_stacks[selected_element], name='EDS results for {}'.format(selected_element), scale=scale, colormap=colormaps[i], opacity=1, blending="additive", rendering="iso")
        new_layer = viewer.add_image(image_stacks[selected_element][ :, a_list[0]:a_list[1], b_list[0]:b_list[1] ], name='EDS results for {}'.format(selected_element), scale=scale, colormap=colormaps[i], opacity=0.5, blending="additive", rendering="mip")
    viewer.scale_bar.visible = True
    
    napari.run()
else:
    print('no elements found')

Found existing translation csv, loading...
Z:/FiB-Data/C3S + EDX/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/SEM Image/\translations_fixed_xy.csv contains 544 lines, while 543 lines were expected
no elements found


In [9]:
#todo

# preprocess the raw eds images - denoising, contrast enhancing and segmentation
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(1,1))
def preprocess_sem_image(img):
    tmp = clahe.apply(cv2.medianBlur(img, 9))

    return clahe.apply( tmp )
    
for i, img in enumerate(loaded_images):
    loaded_images[i] = preprocess_sem_image(img)


# final stack arrangement
corrected_sem_images = create_3D_stack(translation, loaded_images, False)

Create 3D image stack with corrected image translation
[[-1.48157263  2.20504276]
 [ 0.80846742  1.27631032]
 [ 0.04882488  1.23185478]
 ...
 [ 4.21336131  3.229939  ]
 [ 5.30823726  1.86168289]
 [-2.58469469  1.38512177]]
  - allocating 3D image space..
  - translating and denoising images..
image count  (544) equals translations (544 and 544)!


In [11]:
viewer = napari.view_image(corrected_sem_images, name='SEM images', scale=scale)#
viewer.scale_bar.visible = True

napari.run()

C:\Users\Florian Kleiner\AppData\Roaming\Python\Python38\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
